<div style='float: right'><img src='pic/abcplace.png'/></div>
## <div id='paint' />ABCプレース

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
 B B  C 
A......B
B...... 
C...... 
 ......A
 ......C
 ...... 
  BC CA """.split('\n')
nn = len(data)-2

### 問題
* 各行各列にA、B、Cを1つずつ置く。
* 外側のヒントは、最も近い文字を表す。

### 変数
* v：各文字 (1)

### 制約
* ヒントを満たす (2)
* 各行各列に1つ (3)
* 文字は1つ (4)

In [ ]:
m = LpProblem()
v = np.array(addbinvars(nn, nn, 4)) # (1)
def chk(m, c, y, x, dy, dx):
    d = ord(c)-64
    if d > 0:
        for i, k in product(range(1,nn), range(1,4)):
            if k != d:
                for j in range(i):
                    m += v[y+dy*j,x+dx*j,k] <= 1-v[y+dy*i,x+dx*i,d] # (2)
for i in range(nn):
    for k in range(1, 4):
        m += lpSum(v[i,:,k]) == 1 # (3)
        m += lpSum(v[:,i,k]) == 1 # (3)
    chk(m, data[0][i+1], 0, i, 1, 0)
    chk(m, data[nn+1][i+1], nn-1, i, -1, 0)
    chk(m, data[i+1][0], i, 0, 0, 1)
    chk(m, data[i+1][nn+1], i, nn-1, 0, -1)
for i, j in product(range(nn), range(nn)):
    m += lpSum(v[i,j]) == 1 # (4)
%time m.solve()
print('\n'.join(' '.join('.ABC'[i] for i in j)
    for j in np.vectorize(value)(v).dot(range(4)).astype(int)))